In this notebook we will learn how to hedge in quantconnect based on the following example:
We want to invest in the SP500 but without a particular stock called OXY effectively creating a SP499

There are two ways to perform this task:
1. Buy the SP500 and create a short position on OXY with exactly the percentage that OXY has in the SP500
2. Use a Put option hedge: Buy a put option for OXY which insures a 10% loss of this particular stock

## Short Hedging

In order to enable the SP499 we, at first, need to compute the Market Cap of the SP500 and OXY.<br />
The market cap can be computed by:

**EarningReports.BasicAverageShares.ThreeMonths * EarningReports.BasicEPS.TwelveMonths * ValuationRatios.PERatio**

Then we compute the fraction between those two and buy SPY and OXY according to the resulting fractions

In Quantconnect, you can get access to the SP500 (or at least the most similar version to it) by adding **self.Universe.Index.QC500** as Universe.
Additionally, we add **OXY** and **SPY** to our security list

In order to compute the market cap we need to get access to the fundamentals. This can be directly done by: <br />
**self.Securities[data.Key].Fundamentals** <br />
**Important**: data.Fundamentals does not work!<br />
Now we can simply calculate the market cap as defined above and sum those caps for all stocks in the SP500.<br />
In order to store the market cap for OXY we need to check if the current share contains the OXY object. This can be done via: <br /> 
```python
if "OXY" in str(data.Key)
```
Finally we only need to compute the fraction between the OXY market cap and the SP500 total market cap and then buy those equities based on the calculated fractions
(Note: Because we added OXY as an equity it occurs two times in OnData, one time within the QC500 universe and one time as a single slice. Thus we check whether the fraction is < 1 before buying)

In [ ]:
class ShortHedge(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2020, 1, 2)

        self.SetCash(10000)

        self.contractSymbol = None

        self.AddUniverse(self.Universe.Index.QC500)
        
        self.AddEquity("OXY", Resolution.Daily)
        self.AddEquity("SPY", Resolution.Daily)


    def compute_market_cap(self, fundamental):
        market_cap = fundamental.EarningReports.BasicAverageShares.ThreeMonths * \
                     fundamental.EarningReports.BasicEPS.TwelveMonths * \
                     fundamental.ValuationRatios.PERatio
        return market_cap
        
        
    def OnData(self, slice):
        
        if not self.Portfolio.Invested:
            market_cap_sum = 0
            for data in slice:
                
                fundamental = self.Securities[data.Key].Fundamentals
                # If current data object has no fundamental data continue
                if not fundamental:
                    continue
                market_cap = self.compute_market_cap(fundamental)
                
                # If OXY is the current data object, store its market cap
                if "OXY" in str(data.Key):
                    oxy_cap = market_cap
                    self.Debug(f"OXY MARKET CAP: {oxy_cap}")

                # Add the market cap of the current stock to the sp500 cap
                market_cap_sum+=market_cap
            
            # Compute the fraction between oxy and sp500
            relation_oxy_sp500 = oxy_cap / market_cap_sum
            self.Debug(relation_oxy_sp500)
            
            if relation_oxy_sp500 < 1:  # As we added OXY as an equity it occurs twice
                self.SetHoldings("SPY", 1-relation_oxy_sp500)
                self.SetHoldings("OXY", -relation_oxy_sp500)

## Put Option Hedging

We can also hedge our SP500 position with PUT options. <br />
To do so, we can buy a put option with a price 10% below the current market price and thus secure a 10% drop of the OXY share.
To do so, we filter all options with a strike price 20$ below the current market price up to the current market price. Additionally we only want options which have an expiration date at least 200 days in the future.

We then filter the option contracts according to the strike price ( we want to be as close as possible to 10\%) and eventually sort descendingly by expiration date to buy the option which expires latest.
```python
        # Get only the Put options with a strike price of 10-15% below the current price
        contracts_filtered = [contract for contract in chain if contract.Right == 1 and \
                              ((contract.Strike * 1.1) <= oxy_price) and \
                              ((contract.Strike * 1.15) > oxy_price)]

        # # Get the contract with the latest expiration date within a year
        contracts_filtered_sorted = sorted(contracts_filtered, key=lambda x: x.Expiry, reverse=True)[0]
```

In [ ]:
class FilterByType(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2014, 8, 1)
        self.SetEndDate(2015, 8, 1)

        self.SetCash(10000)

        self.contractSymbol = None

        # Subscribe to the google option  
        self.sp500 = self.AddEquity("SPY", Resolution.Daily)
        self.oxy_share = self.AddEquity("OXY", Resolution.Daily)
        oxy_option = self.AddOption("OXY", Resolution.Minute)

        # Hedge up to 365 days and select a strike price from 20 dollar below the current price up to the current price
        oxy_option.SetFilter(-20, 0, timedelta(200), timedelta(365))

        # Plot current OXY price
        price_plot = Chart('Oxy Price')
        price_plot.AddSeries(Series('Price', SeriesType.Line, 0))
        self.AddChart(price_plot)  # Add the Chart to your environment
        self.plot_day = self.Time.day
        
    def OnData(self, slice):

        if not self.Portfolio.Invested:
            self.Debug("Buy SP500")
            self.SetHoldings(self.sp500.Symbol, 0.9)
            
            for chain in slice.OptionChains.Values:
                self.Debug(chain)
                
                # Get the price for OXY share
                for contract in chain:
                    oxy_price = contract.UnderlyingLastPrice
                    break

                # Get only the Put options with a strike price of 10-15% below the current price
                contracts_filtered = [contract for contract in chain if contract.Right == 1 and \
                                      ((contract.Strike * 1.1) <= oxy_price) and \
                                      ((contract.Strike * 1.15) > oxy_price)]
                
                # # Get the contract with the latest expiration date within a year
                contracts_filtered_sorted = sorted(contracts_filtered, key=lambda x: x.Expiry, reverse=True)[0]
                
                
                self.Debug(f"{self.Time}: Oxy Price: {oxy_price} Strike: {contracts_filtered_sorted.Strike} Exipration: {contracts_filtered_sorted.Expiry}")
                self.SetHoldings(contracts_filtered_sorted.Symbol, 0.1)
                return
        
        # Get the price for OXY share for plotting. Only plot once a day.
        if slice.ContainsKey("OXY") and self.Time.day != self.plot_day:
            self.Plot("Oxy Price", "Price", self.oxy_share.Open)
            self.plot_day = self.Time.day
            
            # Additionally check if we want to sell the option
            if self.oxy_share.Open < 80:
                self.Liquidate(self.option)

